In [85]:
from operator import mul

# Tr_k(P) = P + Pi(P) + Pi^2(P) + Pi^3(P) + ... + Pi^{k - 1}, where P \in E(Fq^k)
# Pi(P) = (P.x^q, P.y^q)
def trace(P, k, q, E):
    trace_P = P
    iter_P = P
    for _ in range(1, k):
        x, y = list(iter_P)[0], list(iter_P)[1]
        iter_P = E(x ** q, y ** q)
        trace_P = trace_P + iter_P
    return trace_P

# map element of Fq2 into Fq12
def into_Fq12(e_fq2, beta, F, gen):
    a = beta.polynomial().list()
    if len(a) == 1 :
        a = a + [0]
    e = e_fq2.polynomial().list()
    if len(e) == 1:
        e = e + [0]
    return F(e[0]) + F(e[1]) * (gen ** 6 - F(a[0])) / F(a[1])

# Untwist Map: E_t(Fq^2)[r] --> E(Fq^12)[r]
def untwist(P, beta, F, gen, t_x, t_y, E):
    x, y = list(P)[0], list(P)[1]
    x, y = into_fq12(x, beta, F, gen), into_fq12(y, beta, F, gen)
    return E(x / t_x, y / t_y)

# map elements of Fq12 into Fq2 with critical conditions
def into_Fq2(e_fq12, F, gen):
    coef = e_fq12.polynomial().list()
    zero_coeff = [1 for i in range(12) if ((len(coef) > i) and (i != 0) and (i != 6) and (F(coef[i]) == F(0)))]
    assert(reduce(mul, zero_coeff) == 1)
    
    return (F(coef[0]) + F(coef[6])) + gen * F(coef[6])

# Twist Map: E(Fq^12)[r] --> E_t(Fq^2)[r]
def twist(P, F, gen, t_x, t_y, E_t):
    x, y = list(P)[0] * t_x, list(P)[1] * t_y
    x, y = into_Fq2(x, F, gen), into_Fq2(y, F, gen)
    return E_t(x, y)

# base prime domain which is a large prime number
q = 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787
# short weierstrass equation of elliptic curve: y^2 = x^3 + A x + B
A, B = 0, 4
# base prime field
Fq = GF(q)
# elliptic curve over base prime field E(Fq)
Efq = EllipticCurve(Fq, [A, B])
print("E(Fq): {} \n".format(Efq.formal()))
ord_Efq = Efq.order()
# maximum prime factor of |E(Fq)| = r * cofac_r_Efq
r = 52435875175126190479447740508185965837690552500527637822603658699938581184513
# cofactor against with r on E(Fq), where r | ord(E(Fq)), it can be used to 
# map any point on E(Fq) into the r-torsion subgroup E(Fq)[r]
cofac_r_Efq = ord_Efq // r
# random point from r-torsion subgroup E(Fq)[r]
P = Efq(
    2262513090815062280530798313005799329941626325687549893214867945091568948276660786250917700289878433394123885724147,
    3165530325623507257754644679249908411459467330345960501615736676710739703656949057125324800107717061311272030899084
)
r_P = r * P
assert(r_P == Efq(0))
# embedding degree
k = 12
tr_P = trace(P, k, q, Efq)
assert(tr_P == k * P)
print("P is in G1 of E(Fq)[r]? [r]P = {}, Tr(P) = [k]P = {} \n".format(r_P, tr_P))

## Fq2 = Fq[X] / X^2 - alpha, where alpha = -1
d = 2
alpha = Fq(-1)
X = Fq['X'].gen()
pol2 = X ** d - alpha
assert(pol2.is_irreducible() == True)
Fq2 = GF(q ** d, 'u', modulus = pol2)
u = Fq2.gen()

## Fq12 = Fq2[X] / X^6 - beta, where beta = u + 1, u is the generator of Fq2
d = 6
beta = u + 1
XX = Fq2['XX'].gen()
pol12 = XX ** d - beta
assert(pol12.is_irreducible() == True)

## twist curve E_t(Fq^2) based on E(Fq), where its short weierstrass equation is: y^2 = x^3 + A x + B * beta_t, beta_t equals beta or beta^t
beta_t = beta 
Efq2_t = EllipticCurve(Fq2, [A, B * beta_t])
## find the proper twisted curve E_t(Fq^2), who has a r-torsion subgroup which is isomorphism with E(Fq^12)
if Efq2_t.order() % r != 0:
    beta_t = beta ** 5
    Efq2_t = EllipticCurve(Fq2, [A, B * beta_t])
print("E_t(Fq^2): {}, whose order is divisible by r? {}\n".format(Efq2_t.formal(), Efq2_t.order() % r == 0))

## twist curve E_t(Fq^12)
Fq12_t = Fq2.extension(pol12, 'x')
Efq12_t = Efq2_t.change_ring(Fq12_t)
    
## Fq12 = Fq[X] / X^12 - 2X^6 + 2
Fq12 = GF(p ** k, 'w', modulus = X ** k - 2 * (X ** 6) + 2)
w = Fq12.gen()
## constant parameters of twist/untwist 
beta_t_x = w ** 2
beta_t_y = w ** 3

## make sure g_E2 is in the r-torsion subgroup on Efq2_t
ord_Efq2_t = Efq2_t.order()
assert(ord_Efq2_t % r == 0)
cofac_r_Efq2_t = ord_Efq2_t // r
# g_E2 = Efq2_t(0)
# while g_E2 == Efq2_t(0):
#     b = Efq2_t.random_element()
#     g_E2 = cofac_r_Efq2_t * b
Q_t = Efq2_t([
    [
        1265792444950586559339325656560420460408530841056393412024045461464508512562612331578200132635472221512040207420018,
        12405554917932443612178266677500354121343140278261928092817953758979290953103361135966895680930226449483176258412
    ],
    [
        3186142311182140170664472972219788815967440631281796388401764195993124196896119214281909067240924132200570679195848,
        1062539859838502367600126754068373748370820338894390252225574631210227991825937548921368149527995055326277175720251
    ],
])
assert(r * Q_t == Efq2_t(0))
print("Q_t = {} is in r-torsion subgroup of tweated curve E_t (against with E) which is defined over degree-2 extension field, i.e Q \in E_t(Fq^2)[r] \n".format(Q_t))

## Elliptic curve defined over full extension field E(Fq^12), point Q actually comes from here,
## even though for the purpose of computation efficiency we twist Q to Q_t, and untwist it back 
## to Q after computation is done.
Efq12 = Efq.change_ring(Fq12)
print("E(Fq^12): {} \n".format(Efq12.formal()))
ord_Efq12 = Efq12.order()
assert(ord_Efq12 % r == 0)
cofac_r_Efq12 = ord_Efq12 // r

## untwist: Q_t \in E_t(Fq^2)[r] --> Q \in E(Fq^12)[r]
Q = untwist(Q_t, beta, Fq, w, beta_t_x, beta_t_y, Efq12)
print("Untwist: Q_t = {} \in E_t(Fq^2)[r] --> Q = {} \in E(Fq^12)[r] \n".format(Q_t, Q))

## make sure g_E12 is in the zero-trace subgroup of E(Fq12)[r], namely G2
r_Q = r * Q
assert(r_Q == Efq12(0))
tr_Q = trace(Q, k, q, Efq12)
assert(tr_Q == Efq12(0))
print("Q is in G2 of E(Fq^12)[r]? [r]Q = {}, Tr(Q) = {} \n".format(r_Q, tr_Q))

## make sure it can be twisted back
Q_t_copy = twist(Q, Fq, u, beta_t_x, beta_t_y, Efq2_t)
assert(Q_t_copy == Q_t)
print("Twist: Q = {} \in E(Fq^12) --> Q_t = {} \in E_t(Fq^2)[r] \n".format(Q, Q_t))

E(Fq): Formal Group associated to the Elliptic Curve defined by y^2 = x^3 + 4 over Finite Field of size 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787 

P is in G1 of E(Fq)[r]? [r]P = (0 : 1 : 0), Tr(P) = [k]P = (1222286896477400935964274624782934860723011509633768540961820478257401572228228756549471397130226164370425502139718 : 3357355217442064033078395569520737034813095279228326492367168983467760422795137446808067854265078476101599856027331 : 1) 

E_t(Fq^2): Formal Group associated to the Elliptic Curve defined by y^2 = x^3 + (4*u+4) over Finite Field in u of size 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787^2, whose order is divisible by r? True

Q_t = (12405554917932443612178266677500354121343140278261928092817953758979290953103361135966895680930226449483176258412*u + 12657924449505865593393256565604204604085308410563934120240454614645085125626

In [109]:
import time

## evaluation of double line divisor function
## arithmetics on fields, not on multiplicative group
def double_line(line_point, eval_point, E, phi, reverse = False):
    ######################## arithemtic on finite field of line_point
    ## lambda = 3x^2 / 2y
    (x_L, y_L) = (list(line_point)[0], list(line_point)[1])
    (x_E, y_E) = (list(eval_point)[0], list(eval_point)[1])
    alpha = (3 * x_L^2) / (2 * y_L)
    x_2L = alpha * alpha - 2 * x_L
    y_2L = -y_L - alpha * (x_2L - x_L)
    
    ######################## arithmetic on mixed finite field
    ## x_E, y_E \in F2
    ## y_L, x_L, alpha, x_2L \in F1
    if reverse:
        ## evaluation of slop line l_2T
        e_1 = phi(y_E) - y_L - alpha * (phi(x_E) - x_L)
        ## evaluation of vertical line v_2T
        e_2 = phi(x_E) - x_2L
    else:
        ## evaluation of slop line l_2T
        e_1 = y_E - phi(y_L) - phi(alpha) * (x_E - phi(x_L))
        ## evaluation of vertical line v_2T
        e_2 = x_E - phi(x_2L)

    return E(x_2L, y_2L), e_1, e_2

## evaluation of add line divisor function
## arithmetics on fields, not on multiplicative group
def add_line(line_left_point, line_right_point, eval_point, E, phi, reverse = False):
    ######################## arithemtic on finite field of line_point
    ## lambda = (y2 - y1) / (x2 - x1)
    (x_L, y_L) = (list(line_left_point)[0], list(line_left_point)[1])
    (x_R, y_R) = (list(line_right_point)[0], list(line_right_point)[1])
    (x_E, y_E) = (list(eval_point)[0], list(eval_point)[1])
    alpha = (y_L - y_R) / (x_L - x_R)
    x_LR = alpha * alpha - x_L - x_R
    y_LR = -y_L - alpha * (x_LR - x_L)
    
    ######################## arithmetic on mixed finite field
    ## x_E, y_E \in F2
    ## y_L, x_L, alpha, x_LR \in F1
    if reverse:
        ## evaluation of slop line l_{T + P}
        e_1 = phi(y_E) - y_L - alpha * (phi(x_E) - x_L)
        ## evaluation of vertical line v_{T + P}
        e_2 = phi(x_E) - x_LR
    else:
        ## evaluation of slop line l_{T + P}
        e_1 = y_E - phi(y_L) - phi(alpha) * (x_E - phi(x_L))
        ## evaluation of vertical line v_{T + P}
        e_2 = x_E - phi(x_LR)
    
    return E(x_LR, y_LR), e_1, e_2

## General Miller Loop Entry
def MillerLoop(P, Q, G, q, phi, reverse = False):
    T = P
    f1 = 1
    f2 = 1
    e_bits = [int(i) for i in bin(q)[2:]]
    ## last bit cannot be evaluated, since the slope would be a vertical line
    for i in range(1, len(e_bits)):
        if (i == len(e_bits) - 1) and (e_bits[i] == 0):
            f1 = f1 * (list(Q)[0] - list(T)[0])
            T = 2 * T
            break
        T, e_1, e_2 = double_line(T, Q, G, phi, reverse)
        f1, f2 = (f1 * f1 * e_1, f2 * f2 * e_2)
        if (i == len(e_bits) - 1) and (e_bits[i] == 1):
            f1 = f1 * (list(Q)[0] - list(T)[0])
            T = T + P
            break
        if e_bits[i] == 1:
            T, e_1, e_2 = add_line(T, P, Q, G, phi, reverse)
            f1, f2 = (f1 * e_1, f2 * e_2)
    assert(T == G(0))
    
    return f1 / f2

## Weil Pairing Entry
def WeilPairing(P, Qx, G1, G12, r, phi):
    t0 = time.perf_counter()
    f_rP_Q = MillerLoop(P, Qx, G1, r, phi, False)
    t1 = time.perf_counter()
    f_rQ_P = MillerLoop(Qx, P, G12, r, phi, True)
    t2 = time.perf_counter()
    mu_r = ((-1) ** r) * (f_rP_Q / f_rQ_P)
    print('\n ##[Weil Pairing] Time consuming: t[f(P, Qx)] = {:.3f},  t[f(Qx, P)] = {:.3f}'.format(t1 - t0, t2 - t1))
    
    return mu_r

G1, G2_t, G2 = (Efq, Efq2_t, Efq12)
C1, C2_t, C2 = (cofac_r_Efq, cofac_r_Efq2_t, cofac_r_Efq12)

## P \in G1 of E(Fq)[r], Q_t \in G2 of E_t(Fq^2)[r]
P, Q_t = (C1 * G1.random_element(), C2_t * G2_t.random_element())
assert(r * P == G1(0))
assert(r * Q_t == G2_t(0))

## Q \in G2 of E(Fq^12)[r]
Q = untwist(Q_t, beta, Fq, w, beta_t_x, beta_t_y, G2)
assert(r * Q == G2(0))
assert(trace(Q, 12, q, G2) == G2(0))

####################################### Weil Pairing Testation 
## P is defined over E(Fq)[r], Q is defined over E(Fq^12)[r]
## phi maps Fq to Fq12
phi = Hom(Fq, Fq12)(Fq.gen().minpoly().roots(Fq12)[0][0])
mu_r_weil = WeilPairing(P, Q, G1, G2, r, phi)
## make sure pairing result is in r-torsion subgroup
assert(mu_r_weil ** r == Fq12(1))
#######################################


 ##[Weil Pairing] Time consuming: t[f(P, Qx)] = 0.053,  t[f(Qx, P)] = 0.092
